In [177]:
import logging
import threading
from datetime import datetime, date, timedelta, time
import json
from geojson import LineString, MultiLineString, Feature, Point, FeatureCollection

from vars import pulses, trips, times

heartrate_file = "../data/heart_rate-2019-01-06.json"
pulserecord = {}

with open(heartrate_file) as json_file:  
    data = json.load(json_file)
    for p in data:
        do = datetime.strptime(p['dateTime'], '%m/%d/%y %H:%M:%S')
        myseconds = (do.hour*3600)+(do.minute*60)+(do.second)
        pulserecord[myseconds]=str(p['value']['bpm'])

sortedpulse = dict(sorted(pulserecord.items()))


def time_in_range(start, end, x):
    """Return true if x is in the range [start, end]"""
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end

In [178]:
sortedpulse = dict(sorted(pulserecord.items()))

In [220]:
def gimmeTimedelta(s):
    return int(timedelta(hours=s.hour, minutes=s.minute, seconds=s.second).total_seconds())

def gimmeLongLat(stups):
    # ugh, must reverse lat and lng
    revtups = []
    for tup in stups:
        revtups.append(tup[::-1])
    print(revtups)
    return revtups

def gimmeGeojson(atrip,myloc):
    feature_collection = []
    # ugh, must reverse lat and lng
    revtups = gimmeLongLat(atrip)
    myline = LineString(revtups)
    mylocrev = Point( gimmeLongLat([myloc])[0] )
    mylocrevfeature = Feature(geometry=mylocrev,name="my current point")
    mylinefeature = Feature(geometry=myline,name="my trip")
    feature_collection = [mylocrevfeature,mylinefeature]
    myfeaturecollection = FeatureCollection(feature_collection)
    return myfeaturecollection

def get_curtimeslot(timeslots):
    curtime = datetime.now().time()
    for idx,slot in enumerate(timeslots):
        if time_in_range(slot['start'], slot['end'], curtime):
            myslot = idx
            t1 = gimmeTimedelta(slot['start'])
            t2 = gimmeTimedelta(slot['end'])
            ct = gimmeTimedelta(curtime)
            duration = int(t2-t1)
            numchunks = len(trips[myslot])
            interval = int(duration/numchunks)
            timeintotrip = int(ct-t1)
            for i in range( numchunks ):
                chunkpoint = int(interval * i)
                if (timeintotrip > chunkpoint) and (timeintotrip-chunkpoint < interval):
                    myloc = trips[myslot][i]
            myfeaturecollection = gimmeGeojson(trips[myslot],myloc)
    return(curtime,myslot,duration,timeintotrip,myloc,myfeaturecollection)

In [221]:
curtime,myslot,duration,timeintotrip,myloc,myfeaturecollection = get_curtimeslot(times)

print('curtime:',curtime,'myslot:',myslot,'duration:',duration,'timeintotrip:',timeintotrip,'myloc:',myloc)

[(-76.66676, 39.18099), (-76.66676, 39.18099), (-76.66584, 39.180187)]
[(-76.66584, 39.180187)]
curtime: 15:01:54.348289 myslot: 7 duration: 20981 timeintotrip: 17121 myloc: (39.180187, -76.66584)


In [222]:
myfeaturecollection

{"features": [{"geometry": {"coordinates": [-76.66584, 39.180187], "type": "Point"}, "name": "my current point", "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [[-76.66676, 39.18099], [-76.66676, 39.18099], [-76.66584, 39.180187]], "type": "LineString"}, "name": "my trip", "properties": {}, "type": "Feature"}], "type": "FeatureCollection"}

In [218]:
obj = LineString(trips[myslot])
obj.errors()

In [219]:
obj

{"coordinates": [[39.18099, -76.66676], [39.18099, -76.66676], [39.180187, -76.66584]], "type": "LineString"}